### Prepare raw data

Download html files from site

In [1]:
#!wget -r -k -l 7 -p -E -nc https://povar.ru/ratingCountratingCount
# !rm -rf ./chroma_db

In [2]:
# !apt-get update
# !apt-get install unzip
# !unzip povar_recipes_1.zip
# !unzip povar_recipes_2.zip
# !unzip povar_recipes_3.zip

### Make DataSet

In [3]:
%load_ext autoreload
%autoreload
from utils.utils import make_data_set, read_recipes_pkl, save_recipes_pkl, make_recipes,filter_recipe

%load_ext autoreload
%autoreload
from Recipe import Recipe, RecipesProject

MAIN_DIR = "/home/zaderu/Documents/MLHS/LLM Project/data/povar.ru/recipes/"
FILE_NAME = "/workspace/llm_project/povar_recipes_"

/workspace/recipe_project/llm_venv/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# povar_recipes = make_data_set(MAIN_DIR)
# save_recipes_pkl(povar_recipes, FILE_NAME)

povar_recipes = []
for i in range(1, 4):
    _povar_recipes = read_recipes_pkl(FILE_NAME + str(i) + ".pickle")
    povar_recipes.extend(_povar_recipes)
len(povar_recipes)

Collected 22000 recipes
Collected 22000 recipes
Collected 22349 recipes


66349

In [5]:
recipes_list = make_recipes(povar_recipes)

Total 66349 recipes is processed.


In [6]:
recipes_list = filter_recipe(recipes_list, max_steps = 10, max_min=120, min_rating=4, min_votes=2)
save_recipes_pkl(recipes_list, "./Recipe_final.pickle")

32330 recipes are selected.


In [7]:
recipes_list[0].document.page_content

'Сосиски в лаваше жареные лаваш тонкий, сосиски, сыр твердый, соусЛаваш расстилаем на доске. Чтобы не крошился - можно немного его подержать над паром, станет пластичнее. Смазываем его соусом на ваше усмотрение, посыпаем обильно тертым сыром., На краешек выкладываем сосиску, сворачиваем рулет. У меня было четыре маленьких сосиски, - с разных сторон лаваша начинаем заворачивать, а посередине обрезаем., Обжариваем на масле или на сухой сковороде со всех сторон., Все готово! Приятного аппетита!'

### Create Chroma DB

In [8]:
# Chroma DB
%load_ext autoreload
%autoreload
from build_chroma_db import build_chroma_db, requst_chroma_db
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
vector_store, tokenizer = build_chroma_db(
    recipes_list,
    embedding_model="ai-forever/sbert_large_nlu_ru", #"nomic-ai/nomic-embed-text-v1.5", #"IlyaGusev/saiga_yandexgpt_8b" 
    device=device,
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/workspace/recipe_project/llm_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


39813 chunks are to be loaded to chroma db


### Create Knowledgr Graph

In [9]:
### Knowledge graph
%load_ext autoreload
%autoreload
from build_knowledge_graph import make_tags_list, lemmatize_tags, build_knowledge_graph\
    , lemmatize, lemmatize_sentance, make_one_word_tags_list, enreach_query_with_relative_tags

tags = make_tags_list(recipes_list)

lemmatize_tags(tags)

one_word_tags = make_one_word_tags_list(tags)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
Graph = build_knowledge_graph(recipes_list, tags)

In [11]:
rp = RecipesProject(
    recipes=recipes_list,
    knowledgeGraph=Graph,
    tags=tags,
    vectorStore=vector_store,
    oneWordTags=one_word_tags
)
# rp.add_resipes_list(recipes_list)
# rp.add_knowledge_graph(G)
# rp.add_tags(tags)
# rp.add_one_word_tags(one_word_tags)

### Create LLM

In [25]:
%load_ext autoreload
%autoreload
from build_ollama import build_model, llm_invoke
llm = build_model()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Testing

In [26]:
requests = [
    ["Какой рецепт вегетарианской пасты?"],
    ["Как приготовить блинчики?"],
    ["Что можно приготовить из куриного филе?"],
    ["Как сделать шоколадный торт?"],
    ["Как приготовить рис для суши?"],
    ["Как приготовить суп-пюре из тыквы?"],
    ["Что можно приготовить из картошки и сыра?"],
    ["Как сделать домашний хлеб?"],
    ["Какие десерты можно приготовить без муки?"],
    ["Как приготовить рис с курицей и овощами?"],
    ["Как сделать соус для пасты?"],
    ["Как приготовить морковные котлеты?"],
    ["Что приготовить на ужин за 30 минут?"],
    ["Как сделать домашнюю пиццу?"],
    ["Что можно приготовить из куриного филе, риса и брокколи?"],
    ["Как приготовить сладкий омлет?"],
    ["Как сделать домашнее мороженое без мороженицы?"],
    ["Что можно приготовить для обеда из мяса и риса?"],
    ["Как приготовить крем-суп из грибов?"],
    ["Как приготовить рыбу с картошкой в духовке?"],
    ["Что приготовить для завтрака, если есть яйца, авокадо и помидоры?"],
    ["Как приготовить куриные крылышки на гриле?"],
    ["Что приготовить на праздничный ужин?"],
    ["Как приготовить соус бешамель?"],
    ["Как приготовить кальмары с овощами ?"],
]

### Queries

In [27]:
recipes_dict = {r.id: r for r in rp.recipes}

In [28]:
import random
def final_query(query, n=3):
    query_clean = query.lower().replace("рецепт","").replace("кухня","")
    results_kg = rp.invoke(query_clean, verbose=False)
    if len(results_kg[0]) == 0:
        results_kg = results_kg[1]
    else:
        results_kg = results_kg[0]
    id_kg = [id for id in results_kg]

    results_db = requst_chroma_db(vector_store, tokenizer, query, device, k=100)
    id_db = [r.metadata['id'] for r in results_db]
    
    res_final = []
    for id in results_db:
        if id in id_kg:
            res_final.append(id)
    if len(res_final) < n:
        kg_rest = [id for id in results_kg if id not in id_db]
        res_final.extend(random.sample(kg_rest, k=min([n-len(res_final), len(kg_rest)])))
    if len(res_final) < n:
        db_rest = [id for id in id_db if id not in res_final]
        res_final.extend(db_rest[0:n-len(res_final)])
        
    return id_kg[:n], id_db[:n], res_final[:n]

In [29]:
%load_ext autoreload
%autoreload
from prompt import create_chat_history, prompt, formatted_chat_history, formatted_rag
chat_history = create_chat_history()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
import time
from tqdm import tqdm

log = []

for query in tqdm(requests[:10]):
    log.append([])
    chat_history = create_chat_history()
    for question in query:
        start_time = time.time()
        kg, db, final = final_query(question)
        format_chat_history = formatted_chat_history(chat_history)
        format_rag = formatted_rag([recipes_dict[r].display for r in final])
        question_input = {
            "chat_history": format_chat_history,
            "user_input": question,
            "retrieved_recipes": format_rag
        }
        final_prompt = prompt.format(**question_input)
        llm_prompt_result = llm_invoke(llm, final_prompt)
        llm_prompt_time = time.time() - start_time
        start_time = time.time()
        llm_question_result = llm_invoke(llm, question)
        llm_question_time = time.time() - start_time
        chat_history.add_user_message(question)
        chat_history.add_ai_message(llm_prompt_result)
        log[-1].append({
            'kg':kg,
            'db':db,
            'llm_question':llm_question_result,
            'time_question':llm_question_time,
            'llm_prompt':llm_prompt_result,
            'time_prompt':llm_prompt_time,
        })

100%|██████████| 10/10 [02:05<00:00, 12.60s/it]


In [31]:
for query, ll in zip(requests, log):
    for q, l in zip(query, ll):
        kg = recipes_dict[l['kg'][0]].display
        db = recipes_dict[l['db'][0]].display
        
        print(f"\n\n**Question**: {q}\n\n\t**KG**: {kg}\n\n\t**DB**: {db}\n\n\t**LLM_Prompt**: {l['llm_prompt']}\n\n\t**LLM_Query**: {l['llm_question']}")
    



**Question**: Какой рецепт вегетарианской пасты?

	**KG**: Паста с брокколи и сыром

Количество порций: 4.0.
Ингридиенты
	паста: 400 грамм.
	брокколи: 1 штука.
	сливки: 1-1,5 стакана.
	тертый сыр (смесь): 1 стакан.
	соль, перец: по вкусу.

Способ приготовления
	1. Подготовить продукты..
	2. Отварить пасту согласно инструкции..
	3. Слить воду..
	4. Сливки налить в кастрюлю и подогреть..
	5. К сливкам прибавить порезанные брокколи и варить до готовности..
	6. Пасту добавить в кастрюлю и хорошо перемешать..
	7. Приправить солью, перцем и добавить тертый сыр..
	8. Хорошо смешайте и снимите с огня..
	9. Приятного аппетита!.
Время: 30 мин.


	**DB**: Ленивая ачма

Количество порций: 10.0.
Ингридиенты
	яйцо куриное: 6 штук.
	сыр твердый: 250 грамм.
	брынза или сулугуни: 250 грамм.
	молоко: 1,5 стакана.
	лаваш тонкий: 6 штук.
	чеснок: 1 зубчик.
	соль, перец: по вкусу.

Способ приготовления
	1. Как приготовить ленивую ачму? Для начала возьмите лаваш. Обрежьте его по форме вашей формы для запе

In [31]:
import pickle

with open("log.pickle", 'wb') as handle:
        pickle.dump(log, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
[
    ['Как приготовить карбонару?', 'Что нужно для соуса?'],
    ['Что приготовить детского с мясом?'],
    ['Быстрый рецепт пирога из яблок?'],
    ['Что сделать из кабачков?'],
    ['У меня есть картошка, морковь и духовка. Что мне можно сделать?'],
    ['Дай мне рецепт жареной рыбы с картофелем.'],
    ['Мои дети голодные, а у меня нет времени. Как их накормить?'],
    ['Что нужно для фруктового салата со сладкими фруктами.','Какие фрукты можно использовать вместо дыни?'],
]

In [21]:
chat_history

InMemoryChatMessageHistory(messages=[HumanMessage(content='Как приготовить рис с курицей и овощами?', additional_kwargs={}, response_metadata={}), AIMessage(content='{\n    "assistant": {\n      "message": "Здравствуйте! Мне кажется, вы ищете рецепт курицы, который можно готовить быстро. Я предложу два рецепта, которые готовятся за час. Это курица с рисом и овощами и салат слоеный с печенью. Каждый из них отлично подходит для обеда или ужина. Какой из них понравится вам больше?"\n    }\n  }', additional_kwargs={}, response_metadata={})])

In [22]:
prompt

PromptTemplate(input_variables=['chat_history', 'retrieved_recipes', 'user_input'], input_types={}, partial_variables={}, template='\nВы — опытный кулинарный помощник. Ваша цель — давать полезные, креативные и простые в использовании советы по приготовлению пищи.\nУ вас есть доступ к большой базе рецептов, хранящейся в ChromaDB, которая позволяет вам извлекать соответствующие рецепты и предлагать\nперсонализированные идеи приготовления пищи на основе предпочтений пользователя, доступных ингредиентов, диетических ограничений и стиля приготовления.\n\nИспользуйте историю чата, чтобы сохранять контекст и избегать повторений. Если пользователь упомянул определенные ингредиенты,\nпредпочтения или антипатии ранее в разговоре, учтите это при предоставлении совета.\n\nИспользуйте выбранные рецепты из ChromaDB в качестве руководства для создания индивидуальных предложений. Вы можете комбинировать элементы из разных рецептов или адаптировать их на основе предпочтений пользователя.\n\nЧетко форма

In [24]:
for query in tqdm(requests[:10]):
    chat_history = create_chat_history()
    for question in query:
        start_time = time.time()
        kg, db, final = final_query(question)
        format_chat_history = formatted_chat_history(chat_history)
        format_rag = formatted_rag([recipes_dict[r].display for r in final])
        question_input = {
            "chat_history": format_chat_history,
            "user_input": question,
            "retrieved_recipes": format_rag
        }
        final_prompt = prompt.format(**question_input)
        print(final_prompt)

 10%|█         | 1/10 [00:00<00:03,  2.55it/s]


Вы — опытный кулинарный помощник. Ваша цель — давать полезные, креативные и простые в использовании советы по приготовлению пищи.
У вас есть доступ к большой базе рецептов, хранящейся в ChromaDB, которая позволяет вам извлекать соответствующие рецепты и предлагать
персонализированные идеи приготовления пищи на основе предпочтений пользователя, доступных ингредиентов, диетических ограничений и стиля приготовления.

Используйте историю чата, чтобы сохранять контекст и избегать повторений. Если пользователь упомянул определенные ингредиенты,
предпочтения или антипатии ранее в разговоре, учтите это при предоставлении совета.

Используйте выбранные рецепты из ChromaDB в качестве руководства для создания индивидуальных предложений. Вы можете комбинировать элементы из разных рецептов или адаптировать их на основе предпочтений пользователя.

Четко форматируйте ответы, включая:
1. **Название**: название рецепта или предложения.
2. **Ингредиенты**: перечислите ингредиенты с количеством.
3. **Ин

 20%|██        | 2/10 [00:00<00:02,  2.90it/s]


Вы — опытный кулинарный помощник. Ваша цель — давать полезные, креативные и простые в использовании советы по приготовлению пищи.
У вас есть доступ к большой базе рецептов, хранящейся в ChromaDB, которая позволяет вам извлекать соответствующие рецепты и предлагать
персонализированные идеи приготовления пищи на основе предпочтений пользователя, доступных ингредиентов, диетических ограничений и стиля приготовления.

Используйте историю чата, чтобы сохранять контекст и избегать повторений. Если пользователь упомянул определенные ингредиенты,
предпочтения или антипатии ранее в разговоре, учтите это при предоставлении совета.

Используйте выбранные рецепты из ChromaDB в качестве руководства для создания индивидуальных предложений. Вы можете комбинировать элементы из разных рецептов или адаптировать их на основе предпочтений пользователя.

Четко форматируйте ответы, включая:
1. **Название**: название рецепта или предложения.
2. **Ингредиенты**: перечислите ингредиенты с количеством.
3. **Ин

 30%|███       | 3/10 [00:01<00:02,  2.53it/s]

...


Вы — опытный кулинарный помощник. Ваша цель — давать полезные, креативные и простые в использовании советы по приготовлению пищи.
У вас есть доступ к большой базе рецептов, хранящейся в ChromaDB, которая позволяет вам извлекать соответствующие рецепты и предлагать
персонализированные идеи приготовления пищи на основе предпочтений пользователя, доступных ингредиентов, диетических ограничений и стиля приготовления.

Используйте историю чата, чтобы сохранять контекст и избегать повторений. Если пользователь упомянул определенные ингредиенты,
предпочтения или антипатии ранее в разговоре, учтите это при предоставлении совета.

Используйте выбранные рецепты из ChromaDB в качестве руководства для создания индивидуальных предложений. Вы можете комбинировать элементы из разных рецептов или адаптировать их на основе предпочтений пользователя.

Четко форматируйте ответы, включая:
1. **Название**: название рецепта или предложения.
2. **Ингредиенты**: перечислите ингредиенты с количеством.
3. **Ин

 90%|█████████ | 9/10 [00:03<00:00,  2.21it/s]


Вы — опытный кулинарный помощник. Ваша цель — давать полезные, креативные и простые в использовании советы по приготовлению пищи.
У вас есть доступ к большой базе рецептов, хранящейся в ChromaDB, которая позволяет вам извлекать соответствующие рецепты и предлагать
персонализированные идеи приготовления пищи на основе предпочтений пользователя, доступных ингредиентов, диетических ограничений и стиля приготовления.

Используйте историю чата, чтобы сохранять контекст и избегать повторений. Если пользователь упомянул определенные ингредиенты,
предпочтения или антипатии ранее в разговоре, учтите это при предоставлении совета.

Используйте выбранные рецепты из ChromaDB в качестве руководства для создания индивидуальных предложений. Вы можете комбинировать элементы из разных рецептов или адаптировать их на основе предпочтений пользователя.

Четко форматируйте ответы, включая:
1. **Название**: название рецепта или предложения.
2. **Ингредиенты**: перечислите ингредиенты с количеством.
3. **Ин

100%|██████████| 10/10 [00:04<00:00,  2.32it/s]


Вы — опытный кулинарный помощник. Ваша цель — давать полезные, креативные и простые в использовании советы по приготовлению пищи.
У вас есть доступ к большой базе рецептов, хранящейся в ChromaDB, которая позволяет вам извлекать соответствующие рецепты и предлагать
персонализированные идеи приготовления пищи на основе предпочтений пользователя, доступных ингредиентов, диетических ограничений и стиля приготовления.

Используйте историю чата, чтобы сохранять контекст и избегать повторений. Если пользователь упомянул определенные ингредиенты,
предпочтения или антипатии ранее в разговоре, учтите это при предоставлении совета.

Используйте выбранные рецепты из ChromaDB в качестве руководства для создания индивидуальных предложений. Вы можете комбинировать элементы из разных рецептов или адаптировать их на основе предпочтений пользователя.

Четко форматируйте ответы, включая:
1. **Название**: название рецепта или предложения.
2. **Ингредиенты**: перечислите ингредиенты с количеством.
3. **Ин

In [1]:
ground_trouth = ["""
### 🍝 **Вегетарианская паста с овощами и томатным соусом**  

#### **Ингредиенты**  
- Паста (спагетти или пенне) — 250 г  
- Оливковое масло — 2 ст. ложки  
- Помидоры (консервированные или свежие) — 400 г  
- Чеснок — 2 зубчика (измельчить)  
- Лук — 1 шт. (мелко нарезать)  
- Кабачок — 1 шт. (нарезать кубиками)  
- Перец сладкий — 1 шт. (нарезать полосками)  
- Базилик (свежий или сушёный) — 1 ч. ложка  
- Орегано — 1 ч. ложка  
- Соль и перец — по вкусу  

#### **Инструкция**  
1. **Варка пасты**: Отварите пасту в подсоленной воде до состояния *аль денте*.  
2. **Приготовление соуса**:  
   - В сковороде разогрейте оливковое масло.  
   - Добавьте лук и чеснок, обжаривайте до прозрачности (около 2 минут).  
   - Добавьте перец и кабачок, обжаривайте 5–7 минут.  
   - Влейте помидоры, добавьте базилик, орегано, соль и перец.  
   - Томите на слабом огне 10 минут, периодически помешивая.  
3. **Смешивание**: Добавьте пасту в соус, тщательно перемешайте.  
4. **Подача**: Подавайте горячей, посыпав свежим базиликом или тёртым пармезаном (по желанию).  

#### **Советы/Альтернативы**  
✅ Можно добавить оливки или каперсы для более насыщенного вкуса.  
✅ Кабачок можно заменить на баклажан или брокколи.  
✅ Вместо пасты попробуйте использовать цельнозерновые макароны или безглютеновые варианты.  

""",
"""
### 🥞 **Классические блинчики**  

#### **Ингредиенты**  
- Мука — 150 г  
- Молоко — 300 мл  
- Яйцо — 2 шт.  
- Сахар — 1 ст. ложка  
- Соль — щепотка  
- Растительное масло — 2 ст. ложки  

#### **Инструкция**  
1. **Приготовление теста**:  
   - В миске взбейте яйца с сахаром и солью.  
   - Добавьте молоко, затем постепенно добавляйте муку, постоянно взбивая, чтобы не было комков.  
   - Влейте масло, перемешайте до однородности.  
2. **Жарка**:  
   - Разогрейте сковороду, смажьте небольшим количеством масла.  
   - Вылейте немного теста, равномерно распределяя его по сковороде.  
   - Жарьте 1–2 минуты с каждой стороны до золотистой корочки.  
3. **Подача**: Подавайте с мёдом, джемом или сметаной.  

#### **Советы/Альтернативы**  
✅ Можно добавить ванильный сахар для аромата.  
✅ Если тесто густое — добавьте немного молока.  
✅ Для безглютенового варианта используйте рисовую или овсяную муку.  

""",
"""
### 🍗 **Куриное филе в сливочном соусе**  

#### **Ингредиенты**  
- Куриное филе — 2 шт.  
- Сливки (20–30%) — 200 мл  
- Чеснок — 2 зубчика  
- Сливочное масло — 30 г  
- Соль и перец — по вкусу  
- Петрушка — для украшения  

#### **Инструкция**  
1. Нарежьте куриное филе на небольшие куски.  
2. Разогрейте сливочное масло в сковороде, обжарьте филе до румяной корочки (5–7 минут).  
3. Добавьте измельчённый чеснок, жарьте ещё 1 минуту.  
4. Влейте сливки, посолите и поперчите. Томите на слабом огне 10 минут.  
5. Подавайте, посыпав петрушкой.  

#### **Советы/Альтернативы**  
✅ Можно добавить грибы или шпинат в соус.  
✅ В сливки можно добавить немного тёртого пармезана для насыщенности.  

""",
"""
### 🍫 **Шоколадный торт без выпечки**  

#### **Ингредиенты**  
- Печенье — 200 г  
- Сливочное масло — 100 г  
- Тёмный шоколад — 200 г  
- Сливки (30%) — 150 мл  

#### **Инструкция**  
1. Измельчите печенье в крошку, смешайте с растопленным маслом.  
2. Утрамбуйте в форму, охладите 30 минут.  
3. Растопите шоколад на водяной бане, влейте сливки, перемешайте.  
4. Вылейте на основу из печенья.  
5. Охладите в холодильнике 2 часа.  

#### **Советы/Альтернативы**  
✅ Можно добавить орехи или кокосовую стружку в основу.  
✅ Вместо тёмного шоколада используйте молочный или белый.  

""",
"""
### 🍠 **Картофельные вафли с сыром**  

#### **Ингредиенты**  
- Картофель — 2 шт.  
- Яйцо — 1 шт.  
- Сыр (тёртый) — 100 г  
- Мука — 2 ст. ложки  
- Соль и перец — по вкусу  

#### **Инструкция**  
1. Натрите картофель на мелкой тёрке, отожмите лишний сок.  
2. Смешайте с яйцом, сыром, мукой, солью и перцем.  
3. Выпекайте в вафельнице до золотистой корочки (5–7 минут).  

#### **Советы/Альтернативы**  
✅ Можно добавить зелень (укроп, петрушку).  
✅ Если нет вафельницы — используйте сковороду.  

""",
"""
### 🍞 **Домашний хлеб**  

#### **Ингредиенты**  
- Мука — 500 г  
- Вода — 300 мл  
- Дрожжи — 7 г (сухие)  
- Соль — 1 ч. ложка  
- Сахар — 1 ч. ложка  

#### **Инструкция**  
1. Смешайте дрожжи с тёплой водой и сахаром, оставьте на 10 минут.  
2. Добавьте муку, соль и замесите тесто.  
3. Оставьте в тёплом месте на 1 час.  
4. Выложите тесто в форму, дайте подняться ещё 30 минут.  
5. Выпекайте при 180°C 30 минут.  

#### **Советы/Альтернативы**  
✅ Можно добавить орехи, семечки или оливки в тесто.  
✅ Для хрустящей корочки смажьте верх водой перед выпечкой.  
""",
"""
### 🍚 **Рис с курицей и овощами**  

#### **Ингредиенты**  
- Рис — 200 г  
- Куриное филе — 150 г  
- Морковь — 1 шт.  
- Горошек (замороженный) — 100 г  
- Соевый соус — 2 ст. ложки  

#### **Инструкция**  
1. Отварите рис до готовности.  
2. Обжарьте курицу на сковороде (5–7 минут).  
3. Добавьте морковь и горошек, жарьте 5 минут.  
4. Влейте соевый соус, добавьте рис, перемешайте.  
5. Жарьте ещё 2–3 минуты.  

#### **Советы/Альтернативы**  
✅ Добавьте яйца для версии «жареного риса».  
✅ Можно использовать креветки вместо курицы.  
""",
]

In [46]:
from evaluate import load

bertscore = load("bertscore")
predictions = ["""   - Количество порций: 3
   - Ингредиенты: Баклажан, чеснок, шампиньоны, паста, соль, подсолнечное масло, лук репчатый, черный молотый перец
   - Способ приготовления:
     - Подготовьте ингредиенты для приготовления пасты с баклажанами и грибами.
     - Очистите лук и чеснок, нарежьте мелко и выложите в сковороду с подсолнечным маслом. Помешивая, обжаривайте до прозрачности.
     - Грибы очистите, нарежьте пластинками и добавьте в сковороду. Помешивая обжаривайте ещё 7-10 минут.
     - На другую сковороду выложите очищенные и нарезанные небольшим кубиком баклажаны. Добавьте подсолнечное масло.
     - Обжарьте баклажаны до золотистой корочки.
     - В  кастрюлю с кипящей, подсоленной водой выложите пасту. Варите согласно инструкции на упаковке до готовности. Затем слейте воду.
     - В кастрюлю с пастой добавьте обжаренные баклажаны.
     - Сюда же добавьте обжаренные лук, чеснок и грибы. Посоли и поперчите по вкусу, перемешайте.
     - Сытная, вкусная с грибным ароматом паста готова. Подавайте к столу.
   - Калории: 92 ккал
   - Белки: 2 г
   - Жиры: 5 г
   - Углеводы: 8 г
   - Время: 40 мин""", "general kenobi"]
references = ["""
### 🍝 **Вегетарианская паста с овощами и томатным соусом**  

#### **Ингредиенты**  
- Паста (спагетти или пенне) — 250 г  
- Оливковое масло — 2 ст. ложки  
- Помидоры (консервированные или свежие) — 400 г  
- Чеснок — 2 зубчика (измельчить)  
- Лук — 1 шт. (мелко нарезать)  
- Кабачок — 1 шт. (нарезать кубиками)  
- Перец сладкий — 1 шт. (нарезать полосками)  
- Базилик (свежий или сушёный) — 1 ч. ложка  
- Орегано — 1 ч. ложка  
- Соль и перец — по вкусу  

#### **Инструкция**  
1. **Варка пасты**: Отварите пасту в подсоленной воде до состояния *аль денте*.  
2. **Приготовление соуса**:  
   - В сковороде разогрейте оливковое масло.  
   - Добавьте лук и чеснок, обжаривайте до прозрачности (около 2 минут).  
   - Добавьте перец и кабачок, обжаривайте 5–7 минут.  
   - Влейте помидоры, добавьте базилик, орегано, соль и перец.  
   - Томите на слабом огне 10 минут, периодически помешивая.  
3. **Смешивание**: Добавьте пасту в соус, тщательно перемешайте.  
4. **Подача**: Подавайте горячей, посыпав свежим базиликом или тёртым пармезаном (по желанию).  

#### **Советы/Альтернативы**  
✅ Можно добавить оливки или каперсы для более насыщенного вкуса.  
✅ Кабачок можно заменить на баклажан или брокколи.  
✅ Вместо пасты попробуйте использовать цельнозерновые макароны или безглютеновые варианты.  

""", "general kenobi"]
results = bertscore.compute(predictions=predictions, references=references, lang="ru")

In [47]:
results

{'precision': [0.7606395483016968, 1.0],
 'recall': [0.690859317779541, 1.0],
 'f1': [0.7240720987319946, 1.0],
 'hashcode': 'bert-base-multilingual-cased_L9_no-idf_version=0.3.12(hug_trans=4.49.0)'}